[View in Colaboratory](https://colab.research.google.com/github/UjjwalSaxena/Simple-eye-detection-tests-ML/blob/master/main.ipynb)

In [0]:
# !wget http://parnec.nuaa.edu.cn/xtan/data/datasets/dataset_B_Eye_Images.rar 

In [30]:
# !pip install patool
# !pip install pyunpack
# !mkdir "EyeImages"
!ls -lrt

total 5660
-rw-r--r-- 1 root root 2649411 Oct 27  2013 dataset_B_Eye_Images.rar
drwxr-xr-x 3 root root    4096 Jul 31 16:40 datalab
-rw-r--r-- 1 root root 3130613 Aug  2 16:55 dataset_B_Eye_Images.zip
drwxr-xr-x 3 root root    4096 Aug  2 16:55 EyeImages


In [0]:
# from pyunpack import Archive
# Archive('dataset_B_Eye_Images.rar').extractall('EyeImages')
# import patoolib
# patoolib.extract_archive("dataset_B_Eye_Images.rar", outdir="EyeImages")
# !pip install unrar
# from unrar import rarfile
# rar = rarfile.RarFile('dataset_B_Eye_Images.rar')
# rar.namelist()

In [0]:

# def upload_files():
#   from google.colab import files
#   uploaded = files.upload()
#   for k, v in uploaded.items():
#     open(k, 'wb').write(v)
#   return list(uploaded.keys())

# upload_files()

In [0]:
# Archive('dataset_B_Eye_Images.zip').extractall('EyeImages')

In [0]:
import matplotlib.pyplot as plt
import cv2 
import glob
import numpy as np

In [0]:

def loadImages(path):
  return plt.imread(path)

def getImagesFromFolder(foldername):
  imagePaths = [image_path for image_path in glob.glob(foldername+'/*.jpg')]
#   images= [loadImages(image_path) for image_path in imagePaths]
  return imagePaths

In [0]:
cle= getImagesFromFolder("EyeImages/dataset_B_Eye_Images/closedLeftEyes")
cre= getImagesFromFolder("EyeImages/dataset_B_Eye_Images/closedRightEyes")
ole= getImagesFromFolder("EyeImages/dataset_B_Eye_Images/openLeftEyes")
ore= getImagesFromFolder("EyeImages/dataset_B_Eye_Images/openRightEyes")



In [0]:
from skimage.feature import hog
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features
      
      
def get_histogram_features(img):
  c1= np.histogram(img)
  return c1


  

In [87]:
features_open=[]
features_closed=[]

eye_closed= cle+ cre
eye_open= ole+ ore



for eye_path in eye_closed:
  eye= loadImages(eye_path.strip())
  x = get_hog_features(eye, 11, 16, 2, vis=False, feature_vec=True)
  y = get_histogram_features(eye)
  features_closed.append(np.concatenate((x,y)))

for eye_path in eye_open:
  eye= loadImages(eye_path.strip())
  x = get_hog_features(eye, 11, 16, 2, vis=False, feature_vec=True)
  y = get_histogram_features(eye)
  features_open.append(np.concatenate((x,y)))


/usr/local/lib/python3.6/dist-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [92]:
X = np.vstack((features_open, features_closed)).astype(object)

# print(len(X))
# Define the labels vector
y = np.hstack((np.ones(len(features_open)), np.zeros(len(features_closed))))
# print(len(y))


ValueError: ignored

In [91]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
    
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

# print('Using:',orient,'orientations',pix_per_cell,
#     'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

ValueError: ignored